In [15]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from dotenv import load_dotenv
# //mchatbot env

In [16]:
import os
load_dotenv()
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV')

In [17]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [18]:
extracted_data = load_pdf("../data/")

In [20]:
len(extracted_data)

266

In [21]:
extracted_data[0]

Document(page_content='CCSAP 2017  Book 2   •  Renal/Pulmonary Critical Care 7 Acute Kidney InjuryAcute Kidney Injury\nBy Linda Awdishu, Pharm.D., MAS; and Sheryl E. Wu, Pharm.D., BCPS\nINTRODUCTION\nAcute kidney injury (AKI) results in the abrupt loss of kidney function, \nleading to the retention of waste products, electrolyte disturbances, \nand volume status changes. The term AKI has replaced acute renal \nfailure  because smaller changes in kidney function without overt fail -\nure can result in significant clinical consequences and increased \nmorbidity and mortality.\nChanges in kidney function are detected by a change in biomark -\ners, the most common biomarker being serum creatinine (SCr). \nSerum creatinine is an imperfect biomarker for recognizing AKI, given \nthat an increase in SCr often lags (48–72 hours) behind the onset of \ninjury. In addition, SCr is not in a steady-state condition in critically \nill patients, leading to inaccurate estimates of glomerular filtration

In [5]:
extracted_data[0],extracted_data[1],

(Document(page_content='CCSAP 2017  Book 2   •  Renal/Pulmonary Critical Care 7 Acute Kidney InjuryAcute Kidney Injury\nBy Linda Awdishu, Pharm.D., MAS; and Sheryl E. Wu, Pharm.D., BCPS\nINTRODUCTION\nAcute kidney injury (AKI) results in the abrupt loss of kidney function, \nleading to the retention of waste products, electrolyte disturbances, \nand volume status changes. The term AKI has replaced acute renal \nfailure  because smaller changes in kidney function without overt fail -\nure can result in significant clinical consequences and increased \nmorbidity and mortality.\nChanges in kidney function are detected by a change in biomark -\ners, the most common biomarker being serum creatinine (SCr). \nSerum creatinine is an imperfect biomarker for recognizing AKI, given \nthat an increase in SCr often lags (48–72 hours) behind the onset of \ninjury. In addition, SCr is not in a steady-state condition in critically \nill patients, leading to inaccurate estimates of glomerular filtratio

In [23]:
extracted_data[2].metadata,extracted_data[200].metadata

({'source': '..\\data\\aki.pdf', 'page': 2},
 {'source': '..\\data\\Kidney-book-in-English.pdf', 'page': 180})

In [25]:
extracted_data[200].metadata,extracted_data[200].page_content

({'source': '..\\data\\Kidney-book-in-English.pdf', 'page': 180},
 'contrast agents, maintaining adequate hydration with IV fluids, and\nadministratio n of sodium bicarbonate and acetylcysteine.\n4.Other drugs\nOther common drugs that can damage kidneys are certain antibiotics,\nanticancer therapy , antituberculosis drugs etc.\n5.Other medicines\n•The popular belief that all natural medicines (Aurvedic medicines,\nChinese herbs etc.) and dietary supplements are harmless is wrong.\n•Certain medicines of these groups contain heavy metals and toxic\nsubstances which can cause damage to the kidney .\n•Use of certain medicines of these groups can be dangerous in patients\nwith kidney failure.\n•Certain drugs with high potassium content can be lethal in kidney\nfailure.\nIt is a wrong belief that all natural medicines\nare always safe for the kidney .CHP. 21.   Kidney and Drugs 161.\nFree!!! 200+ Paged Kidney Book in 35+ Languages \n      Visit: www.KidneyEducation.com')

In [26]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [27]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 1109


In [28]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [29]:
embeddings = download_hugging_face_embeddings()

In [30]:
query_emb = embeddings.embed_query("write  table of contents of this book with name of book")
print("Length", len(query_emb))

Length 384


In [31]:
text_chunks[0]

Document(page_content='CCSAP 2017  Book 2   •  Renal/Pulmonary Critical Care 7 Acute Kidney InjuryAcute Kidney Injury\nBy Linda Awdishu, Pharm.D., MAS; and Sheryl E. Wu, Pharm.D., BCPS\nINTRODUCTION\nAcute kidney injury (AKI) results in the abrupt loss of kidney function, \nleading to the retention of waste products, electrolyte disturbances, \nand volume status changes. The term AKI has replaced acute renal \nfailure  because smaller changes in kidney function without overt fail -', metadata={'source': '..\\data\\aki.pdf', 'page': 0})

In [32]:
#Initializing the Pinecone
pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)

index_name="mchatbot"

# #Creating Embeddings for Each of The Text Chunks & storing in pinecone vector store database
docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [34]:
#If we already have an index we can load it like this
docsearch=Pinecone.from_existing_index(index_name, embeddings)

query = "write all the table of contents of this book"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs[0])


Result page_content='How to use the book?\nThis book is into two parts\nPart 1:\nBasic details about kidney and prevention of kidney diseases are\nnarrated. Each and every individual is advised to read this part of the\nbook. The information provided can make a difference, as it prepares\na lay man for early detection and prevention of kidney diseases.\nPart 2:\nOne can read this part as per one’ s curiosity and necessity .\n•Information about major kidney diseases, and its symptoms,'


In [35]:
print("Result:  \n", docs[0].page_content)

Result:  
 How to use the book?
This book is into two parts
Part 1:
Basic details about kidney and prevention of kidney diseases are
narrated. Each and every individual is advised to read this part of the
book. The information provided can make a difference, as it prepares
a lay man for early detection and prevention of kidney diseases.
Part 2:
One can read this part as per one’ s curiosity and necessity .
•Information about major kidney diseases, and its symptoms,


In [36]:
toc=str(docs[0].page_content).split('\n')
print(len(toc))
for i in range(len(toc)):
    print(toc[i])

10
How to use the book?
This book is into two parts
Part 1:
Basic details about kidney and prevention of kidney diseases are
narrated. Each and every individual is advised to read this part of the
book. The information provided can make a difference, as it prepares
a lay man for early detection and prevention of kidney diseases.
Part 2:
One can read this part as per one’ s curiosity and necessity .
•Information about major kidney diseases, and its symptoms,


In [37]:
toc[7],toc[4]

('Part 2:',
 'narrated. Each and every individual is advised to read this part of the')

In [38]:
query = "who is the author of this book"
docs=docsearch.similarity_search(query, k=3)

print("Result", docs[0].page_content)

Result This book is dedicated to all patients with
kidney disease and their families.


In [39]:
docs

[Document(page_content='This book is dedicated to all patients with\nkidney disease and their families.'),
 Document(page_content='How to use the book?\nThis book is into two parts\nPart 1:\nBasic details about kidney and prevention of kidney diseases are\nnarrated. Each and every individual is advised to read this part of the\nbook. The information provided can make a difference, as it prepares\na lay man for early detection and prevention of kidney diseases.\nPart 2:\nOne can read this part as per one’ s curiosity and necessity .\n•Information about major kidney diseases, and its symptoms,'),
 Document(page_content='family but also to those at the risk of developing kidney disease.  And,\nindeed, it will also be of educational importance for all those individuals\nwho value awareness. Medical students, doctors, paramedicals are sure\nto find this book a handy reference guide.\nWe are thankful to Dr Gaurang and Dr . Susmita Dave,\nMr. Ayaazkhan Babi and Neha Babi for their valuable su

## Llama2 model usage for chat

In [40]:
prompt_template="""
you are a powerful assistant expert in kidney desease .
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:"""

In [41]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [42]:
llm=CTransformers(model="../Model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [43]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [44]:
user_input="What is acute kidney injury"
result=qa({"query": user_input})
print("Response : ", result["result"])

Response :  
Acute kidney injury (AKI) is a condition where there is a sudden loss of kidney function, leading to the retention of waste products, electrolyte disturbances, and volume status changes. It can be caused by various factors such as reduced blood supply to the kidneys due to dehydration, blood loss, burns, or fall in blood pressure. AKI is usually temporary and reversible, but it can lead to chronic kidney disease if not treated promptly.


In [ ]:
result

{'query': 'What is acute kidney injury',
 'result': ' Acute kidney injury (AKI) results in the abrupt loss of kidney function, leading to the retention of waste products, electrolyte disturbances, and volume status changes.',
 'source_documents': [Document(page_content='CCSAP 2017  Book 2   •  Renal/Pulmonary Critical Care 7 Acute Kidney InjuryAcute Kidney Injury\nBy Linda Awdishu, Pharm.D., MAS; and Sheryl E. Wu, Pharm.D., BCPS\nINTRODUCTION\nAcute kidney injury (AKI) results in the abrupt loss of kidney function, \nleading to the retention of waste products, electrolyte disturbances, \nand volume status changes. The term AKI has replaced acute renal \nfailure  because smaller changes in kidney function without overt fail -'),
  Document(page_content='CONCLUSION\nAcute kidney injury can be caused by several conditions and \nshould be considered a syndrome rather than an injury alone \nbecause of the many complications and effects on other \norgans. Pharmacists are uniquely positioned 

In [ ]:
# while True:
#     user_input=input(f"Input Prompt:")
#     result=qa({"query": user_input})
#     print("Response : ", result["result"])
#     if user_input=='exit':
#         break